# SDialog dependencies

In [ ]:
# Setup the environment depending on weather we are running in Google Colab or Jupyter Notebook
import os
from IPython import get_ipython

if "google.colab" in str(get_ipython()):
    print("Running on CoLab")

    # Installing sdialog
    !git clone https://github.com/qanastek/sdialog.git
    %cd sdialog
    %pip install -e .
    %cd ..
else:
    print("Running in Jupyter Notebook")
    # Little hack to avoid the "OSError: Background processes not supported." error in Jupyter notebooks"
    get_ipython().system = os.system

## Local installation

Create a `.venv` using the root `requirement.txt` file and Python `3.11.14`

In [ ]:
from sdialog import Dialog
from IPython.display import Audio, display

# Load an existing dialogue

In order to run the next steps in a fast manner, we will start from an existing dialog generated using previous tutorials:

In [ ]:
path_dialog = "../tests/data/demo_dialog_doctor_patient.json"

if not os.path.exists(path_dialog) and not os.path.exists("./demo_dialog_doctor_patient.json"):
    !wget https://raw.githubusercontent.com/qanastek/sdialog/refs/heads/main/tests/data/demo_dialog_doctor_patient.json
    path_dialog = "./demo_dialog_doctor_patient.json"

original_dialog = Dialog.from_file(path_dialog)
original_dialog.print()

# Tutorial 17: Impulse response and recording devices simulation

In [ ]:
from sdialog.audio.dialog import AudioDialog
from sdialog.audio.pipeline import AudioPipeline

Convert the original dialog into a audio enhanced dialog

In [ ]:
dialog: AudioDialog = AudioDialog.from_dialog(original_dialog)

In [ ]:
from sdialog.audio.impulse_response_database import HuggingFaceImpulseResponseDatabase
impulse_response_database = HuggingFaceImpulseResponseDatabase("sdialog/impulse-responses")

In [ ]:
import scaper
DATA_PATH = "./dscaper_data_impulse_response"
os.makedirs(DATA_PATH, exist_ok=True)
dsc = scaper.Dscaper(dscaper_base_path=DATA_PATH)

In [ ]:
os.makedirs("./audio_outputs_impulse_response", exist_ok=True)
audio_pipeline = AudioPipeline(
    dir_audio="./audio_outputs_impulse_response",
    dscaper=dsc,
    impulse_response_database=impulse_response_database
)

In [ ]:
audio_pipeline.populate_dscaper(["sdialog/background","sdialog/foreground"])

Now let's generate a medical room it will be enough and display it's shape and content:

In [ ]:
from sdialog.audio.room import DirectivityType
from sdialog.audio.utils import SourceVolume, SourceType
from sdialog.audio.jsalt import MedicalRoomGenerator, RoomRole

In [ ]:
room = MedicalRoomGenerator().generate(args={"room_type": RoomRole.EXAMINATION})
img = room.to_image()
display(img)
img.save("room.png")

In [ ]:
from sdialog.audio.room import SpeakerSide, Role, RoomPosition

In [ ]:
room.place_speaker_around_furniture(speaker_name=Role.SPEAKER_1, furniture_name="desk", max_distance=1.0, side=SpeakerSide.FRONT)
room.place_speaker_around_furniture(speaker_name=Role.SPEAKER_2, furniture_name="desk", max_distance=1.0, side=SpeakerSide.BACK)

You can see the new positions of the speakers:

In [ ]:
img = room.to_image()
display(img)

Now let's change the microphone directivity to point to something else:

In [ ]:
room.set_directivity(direction=DirectivityType.OMNIDIRECTIONAL)

In [ ]:
from sdialog.audio.processing import RecordingDevice

In [ ]:
dialog: AudioDialog = audio_pipeline.inference(
    dialog,
    environment={
        "room": room,
        "background_effect": "white_noise",
        "foreground_effect": "ac_noise_minimal",
        "foreround_effect_position": RoomPosition.TOP_RIGHT,
        "source_volumes": {
            SourceType.ROOM: SourceVolume.HIGH,
            SourceType.BACKGROUND: SourceVolume.VERY_LOW
        },
        "kwargs_pyroom": {
            "ray_tracing": True,
            "air_absorption": True
        }
    },
    do_step_1=True,
    do_step_2=True,
    do_step_3=True,
    dialog_dir_name="demo_impulse_response",
    room_name="my_room_demo_shure",
    re_sampling_rate=16000,
    recording_device=RecordingDevice.SHURE_SM57
)

In [ ]:
print("-"*25)
print("Step 1:")
print("-"*25)
display(Audio(dialog.audio_step_1_filepath, autoplay=False))
print("-"*25)
print("Step 2:")
print("-"*25)
display(Audio(dialog.audio_step_2_filepath, autoplay=False))
print("-"*25)
print("- Room Configurations")
print("-"*25)
for config_name in dialog.audio_step_3_filepaths:
    print(f"> Room Configuration: {config_name}")
    print("Original audio:")
    display(Audio(dialog.audio_step_3_filepaths[config_name]["audio_path"], autoplay=False, rate=16000))
    print("#"*10)
    print("Processed audio:")
    print("#"*10)
    for _rd in dialog.audio_step_3_filepaths[config_name]["audio_paths_post_processing"]:
        display(Audio(dialog.audio_step_3_filepaths[config_name]["audio_paths_post_processing"][_rd], autoplay=False, rate=16000))

You can also perform microphone simulation by using the `to_audio` functions:

In [ ]:
my_audio_dialog = original_dialog.to_audio(
    do_step_1=True,
    do_step_2=True,
    do_step_3=True,
    dir_audio="./audio_outputs_impulse_response",
    dialog_dir_name="demo_impulse_response_to_audio",
    room_name="my_room_demo_shure_to_audio_3",
    recording_device=RecordingDevice.SENNHEISER_E906
)

In [ ]:
my_audio_dialog = original_dialog.to_audio(
    do_step_1=False,
    do_step_2=False,
    do_step_3=True,
    dir_audio="./audio_outputs_impulse_response",
    dialog_dir_name="demo_impulse_response_to_audio",
    room_name="my_room_demo_shure_to_audio_3",
    recording_device=RecordingDevice.SHURE_SM57
)

In [ ]:
my_audio_dialog = original_dialog.to_audio(
    do_step_1=True,
    do_step_2=True,
    do_step_3=True,
    dir_audio="./audio_outputs_impulse_response",
    dialog_dir_name="demo_impulse_response_to_audio",
    room_name="my_room_demo_shure_to_audio_4",
    recording_device=RecordingDevice.SHURE_SM57
)

In [ ]:
print("-"*25)
print("Step 1:")
print("-"*25)
display(Audio(my_audio_dialog.audio_step_1_filepath, autoplay=False))
print("-"*25)
print("Step 2:")
print("-"*25)
display(Audio(my_audio_dialog.audio_step_2_filepath, autoplay=False))
print("-"*25)
print("- Room Configurations")
print("-"*25)
for config_name in my_audio_dialog.audio_step_3_filepaths:
    print(f"> Room Configuration: {config_name}")
    print("Original audio:")
    display(Audio(my_audio_dialog.audio_step_3_filepaths[config_name]["audio_path"], autoplay=False, rate=16000))
    print("#"*10)
    print("Processed audio:")
    print("#"*10)
    for _rd in my_audio_dialog.audio_step_3_filepaths[config_name]["audio_paths_post_processing"]:
        display(Audio(my_audio_dialog.audio_step_3_filepaths[config_name]["audio_paths_post_processing"][_rd], autoplay=False, rate=16000))